In [369]:
import pandas as pd
import argparse
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm



In [370]:
EPOCH = 100
LR = 0.0001

device = 'cpu'
if torch.cuda.is_available(): # 若想使用 cuda 且可以使用 cuda
    device = 'cuda'

In [371]:
data=pd.read_csv('train_data.csv')
print(data.shape)
data.head()


(82, 3)


,日期,備轉容量(萬瓩),備轉容量率(%)
0,2022/01/01,334.5,13.04
1,2022/01/02,364.1,14.05
2,2022/01/03,318.4,10.62
3,2022/01/04,262.0,8.64
4,2022/01/05,251.2,8.25


In [372]:
def getData(df, column, train_size):

    train_data = np.array(df[column][:-train_size])
    test_data = np.array(df[column][-train_size:])

    return train_data, test_data


In [373]:
train_data, test_data = getData(data,'備轉容量(萬瓩)',7)
scaler = MinMaxScaler(feature_range=(-1, 1))
# Normalization
train_data = scaler.fit_transform(train_data.reshape(-1,1))
train_data = torch.FloatTensor(train_data).view(-1)
# test_data = scaler.fit_transform(test_data.reshape(-1,1))
# test_data = torch.FloatTensor(test_data).view(-1)
print(test_data)

input=[]
for i in range(len(train_data)-7):
    input.append((train_data[i:7+i],train_data[i+7:i+7+1]))

# print(input)


[271.2 442.4 267.5 305.  275.1 373.5 371.7]


### LSTM Model

In [374]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]



In [375]:
model=LSTM()
# model.to(device)
loss=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=LR)
# print(model)

### Train Model

In [376]:
for i in range(EPOCH):
    
    model.train()

    for seq, labels in input:
        # seq, labels = seq.to(device), labels.to(device)
        
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

        y_pred = model(seq)
        # print(y_pred)

        single_loss = loss(y_pred, labels)
        single_loss.backward()
        optimizer.step()
        # break

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')


epoch:   1 loss: 0.09595752
epoch:  26 loss: 0.01288883
epoch:  51 loss: 0.01168325
epoch:  76 loss: 0.01088244
epoch:  99 loss: 0.0103191305


In [377]:
test_input=train_data[-7:].tolist()
# print(model(test_input[-7:]).item())
model.eval()

for i in range(7):
    with torch.no_grad():
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        test_input.append(model(torch.FloatTensor(test_input[-7:])).item())

# print(test_input[:])
    
    

In [378]:
actual_result=scaler.inverse_transform(np.array(test_input[-7:]).reshape(-1,1))
print(test_data)
print(actual_result.reshape(7,))

[271.2 442.4 267.5 305.  275.1 373.5 371.7]
[309.58768616 311.02575073 313.80161438 316.99486694 318.93015442
 319.98020325 320.89013062]
